In [ ]:
import pandas as pd # this is importing a library that could read a csv file
pd.set_option('display.max_colwidth', None) # Set the option to display the full column width to avoid the truncave截尾. Show the full twit.
Iowa = pd.read_csv('/content/Iowa Polling Places.csv') #open the dataset. Click the file and click "rename" to copy its full name and paste it there.
print(Iowa.head(5))

   Unnamed: 0 election_date state county_name jurisdiction jurisdiction_type  \
0           0     11/3/2020    IA       Adair        Adair            county   
1           1     11/3/2020    IA       Adair        Adair            county   
2           2     11/3/2020    IA       Adair        Adair            county   
3           3     11/3/2020    IA       Adair        Adair            county   
4           4     11/3/2020    IA       Adair        Adair            county   

  precinct_id   precinct_name  polling_place_id location_type  ...  \
0         1NW       1NW ADAIR               NaN  election_day  ...   
1         2NE      2NE STUART               NaN  election_day  ...   
2         3SW  3SW FONTANELLE               NaN  election_day  ...   
3         4SE      4SE ORIENT               NaN  election_day  ...   
4         5GF  5GF GREENFIELD               NaN  election_day  ...   

                                   address notes  source source_date  \
0          320 AUDUBON ST,

In [ ]:
print(Iowa.columns) #Show the name of all the variables.
fieldnames = Iowa.columns

Index(['Unnamed: 0', 'election_date', 'state', 'county_name', 'jurisdiction',
       'jurisdiction_type', 'precinct_id', 'precinct_name', 'polling_place_id',
       'location_type', 'name', 'address', 'notes', 'source', 'source_date',
       'source_notes', 'StreetAddress', 'City', 'StateZip', 'State',
       'ZipCode'],
      dtype='object')


In [ ]:
#Split the addresses into separate columns
Iowa[['StreetAddress', 'City', 'StateZip']] = Iowa['address'].str.split(', ', expand=True)

In [ ]:
#Split state and Zip code into separate columns
Iowa[['State', 'ZipCode']] = Iowa['StateZip'].str.split(' ', expand=True)

In [ ]:
#drop column 'StateZip'
Iowa.drop(columns=['StateZip'], inplace=True)

In [ ]:
#Open category dictionary
categorize = pd.read_csv(r'/content/Religious Buildings Dictionary.csv') #open the dictionary
print(categorize.head(10)) #check first 10 rows

            category       word
0  ReligiousBuilding     CHURCH
1  ReligiousBuilding  SYNAGOGUE
2  ReligiousBuilding     MOSQUE
3  ReligiousBuilding  CHRISTIAN
4  ReligiousBuilding  METHODIST
5  ReligiousBuilding   LUTHERAN
6  ReligiousBuilding       ZION
7  ReligiousBuilding   PASTORAL
8  ReligiousBuilding     PARISH
9  ReligiousBuilding      PRESB


In [ ]:
#Create a function to lookup and assign categories
def assign_category(text):
    matching_categories = categorize[categorize['word'].apply(lambda word: str(word) in text)] #this is a function telling that for words in text, see if each word matches with word in the dictionary

    if not matching_categories.empty: #when there is a match
        return ', '.join(matching_categories['category']) #list the category name. When there are more than two matches, put category names joined by ','
    else:
        return 'NonReligious' # if there is no match

In [ ]:
Iowa['category'] = Iowa['name'].apply(assign_category) #make another column, 'category' containing list of categories associated with name

In [ ]:
Iowa['category'].value_counts()

category
NonReligious                                               1257
ReligiousBuilding, ReligiousBuilding                        247
ReligiousBuilding                                           167
ReligiousBuilding, ReligiousBuilding, ReligiousBuilding      10
Name: count, dtype: int64

In [ ]:
#Split the 'category' column by comma and create a new DataFrame with each category in a separate row
categories_split = Iowa['category'].str.split(', ').explode().reset_index(drop=True) #explode to check all the categories associated with the dataset
unique_categories = categories_split.unique() #only retain unique categories
print(unique_categories) #print unique categories found in the dataset

['NonReligious' 'ReligiousBuilding']


In [ ]:
Iowa['category'] = Iowa['category'].astype(str) #make sure the category column is in string
categories_split = Iowa['category'].str.split(', ') # Split the 'category' column by comma (', ') and create binary columns
print(categories_split) #look at the results

for new_column in unique_categories:
    Iowa[new_column] = categories_split.apply(lambda x: x.count(new_column)) #create columns and count the number of the category in the list

Iowa.fillna(0, inplace=True) # Fill NaN values with 0 in the binary columns

0                               [NonReligious]
1                               [NonReligious]
2                               [NonReligious]
3       [ReligiousBuilding, ReligiousBuilding]
4                               [NonReligious]
                         ...                  
1676                            [NonReligious]
1677                            [NonReligious]
1678                            [NonReligious]
1679                            [NonReligious]
1680                            [NonReligious]
Name: category, Length: 1681, dtype: object


In [ ]:
Iowa.columns

Index(['Unnamed: 0', 'election_date', 'state', 'county_name', 'jurisdiction',
       'jurisdiction_type', 'precinct_id', 'precinct_name', 'polling_place_id',
       'location_type', 'name', 'address', 'notes', 'source', 'source_date',
       'source_notes', 'StreetAddress', 'City', 'State', 'ZipCode', 'category',
       'NonReligious', 'ReligiousBuilding'],
      dtype='object')

In [ ]:
def generate_type(df):
    religious = 1
    nonreligious = 2

    # Generate 'building type' based on conditions
    df['BuildingType'] = None  # Initialize new column with None

    # Update 'building type' based on conditions
    df.loc[df['ReligiousBuilding'] > 0, 'BuildingType'] = 'Religious'
    df.loc[df['NonReligious'] > 0, 'BuildingType'] = 'NonReligious'

    return df

Iowa = generate_type(Iowa)

In [ ]:
Iowa.columns

Index(['Unnamed: 0', 'election_date', 'state', 'county_name', 'jurisdiction',
       'jurisdiction_type', 'precinct_id', 'precinct_name', 'polling_place_id',
       'location_type', 'name', 'address', 'notes', 'source', 'source_date',
       'source_notes', 'StreetAddress', 'City', 'State', 'ZipCode', 'category',
       'NonReligious', 'ReligiousBuilding', 'BuildingType'],
      dtype='object')

In [ ]:
Iowa['BuildingType'].value_counts()

BuildingType
NonReligious    1257
Religious        424
Name: count, dtype: int64

In [ ]:
#Save data to csv file.
Iowa.to_csv("Iowa Processed.csv")